In [1]:
import openai
from pathlib import Path
import json
import llm2geneset
import time
import pandas as pd

In [2]:
import re

def clean_elements(array):
    cleaned_array = []
    for element in array:
        # Use regular expression to remove (GO:xxx) substring,  R-HSA-xxx substrings, and WPxxx substrings
        cleaned_element = re.sub(r'\s*\(GO:\d+\)\s*|\s*R-HSA-\d+\s*|\s*WP\d+\s*', '', element)
        cleaned_array.append(cleaned_element)
    return cleaned_array

In [3]:
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human", 
             "GO_Biological_Process_2023",
             "GO_Molecular_Function_2023",
             "GO_Cellular_Component_2023"]
#lib_names = ["KEGG_2021_Human"]
#lib_names = ["KEGG_2021_Human", 
#             "Reactome_2022", 
#             "WikiPathway_2023_Human"]

In [4]:
# Generate table for some basic stats on gene set libraries used.
geneset_sz = []
for lib_name in lib_names:
    (descr, genes) = llm2geneset.read_gmt("libs_human/gmt/" + lib_name + ".txt")
    x = [{"lib_name": lib_name, "descr": d, "num_genes": len(g)} for (d,g) in zip(descr,genes)]
    print(len(genes))
    geneset_sz.extend(x)
df = pd.DataFrame(geneset_sz)
df.to_csv("geneset_sz.tsv",sep="\t",index=None)

320
1818
801
5407
1147
474


In [ ]:
aclient = openai.AsyncClient()
models = ["gpt-3.5-turbo-0125", "gpt-4o-2024-05-13"]
# "gpt-4-turbo-2024-04-09" has garbage random text, doesn't generat correctly using
# the async API
# models = ["gpt-4-turbo-2024-04-09"]
for model in models:
    for lib_name in lib_names:
        (descr, genes) = llm2geneset.read_gmt("libs_human/gmt/" + lib_name + ".txt")
        # Generate cleaned version of gene set description w/o identifiers.
        descr_cleaned = clean_elements(descr)
        
        # Generate genes, and keep track of time it takes (seconds).
        start_time = time.time()
        llm_genes = await llm2geneset.get_genes(aclient, descr_cleaned, model=model)
        end_time = time.time()
        gen_time = end_time - start_time
        gen_res = {}

        # Assemble and save generation results.
        gen_res["lib_name"] = lib_name
        gen_res["model"] = model
        gen_res["gen_time"] = gen_time
        gen_res["descr"] = descr
        gen_res["descr_cleaned"] = descr_cleaned
        gen_res["curated_genesets"] = genes
        gen_res["llm_genesets"] = llm_genes
    
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1817/1818 [00:36<00:02,  2.49s/it]

retrying
No code blocks found
List all the known genes directly and indirectly involved in the following biological process or cellular component """Platelet Activation, Signaling And Aggregation""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """Platelet Activation, Signaling And Aggregation""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {"gene": "ADRA2A"},
    {"gene": "ADRA2B"},
    {"gene": "ADRB1"},
    {"gene": "ADRB2"},
    {"gene": "ADRB3"},
    {"gene": "AKT1"},
    {"gene": "AKT2"},
    {"gene": "AKT3"},
    {"gene": "ARHGEF1"},
    {"gene": "CALM1"},
    {

 93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊         | 742/801 [00:08<00:01, 46.89it/s]

retrying
'gene'
List all the known genes directly and indirectly involved in the following biological process or cellular component """Glycosphingolipid Metabolism""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """Glycosphingolipid Metabolism""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {
        "gene": "ABCA1"
    },
    {
        "gene": "ABCB11"
    },
    {
        "gene": "ABCB4"
    },
    {
        "gene": "ABCG5"
    },
    {
        "gene": "ABCG8"
    },
    {
        "gene": "ASAH1"
    },
    {
        "gene": "ASAH2"
    },
    {
        "gene": "B4G

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 800/801 [00:22<00:01,  1.60s/it]

...

 12%|██████████████▌                                                                                                           | 217/1818 [00:05<00:14, 110.95it/s]

retrying
No code blocks found
List all the known genes directly and indirectly involved in the following biological process or cellular component """Apoptosis""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """Apoptosis""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {"gene": "TP53"},
    {"gene": "BAX"},
    {"gene": "BCL2"},
    {"gene": "CASP3"},
    {"gene": "CASP8"},
    {"gene": "CASP9"},
    {"gene": "FAS"},
    {"gene": "FASLG"},
    {"gene": "BID"},
    {"gene": "BCL2L1"},
    {"gene": "BIK"},
    {"gene": "BAD"},
    {"gene": "BIRC5"},
    {"gene": "DIABLO"}

 74%|█████████████████████████████████████████████████████████████████████████████████████████▉                               | 1351/1818 [00:13<00:03, 143.49it/s]

retrying
'gene'
List all the known genes directly and indirectly involved in the following biological process or cellular component """SUMO Is Proteolytically Processed""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """SUMO Is Proteolytically Processed""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {"gene": "SENP1"},
    {"gene": "SENP2"},
    {"gene": "SENP3"},
    {"gene: "SENP5"},
    {"gene": "SENP6"},
    {"gene": "SENP7"},
    {"gene": "UBA2"},
    {"gene": "SAE1"},
    {"gene": "UBC9"},
    {"gene": "SUMO1"},
    {"gene": "SUMO2"},
    {"gene": "SUMO3"},
    

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 783/801 [00:13<00:01, 14.16it/s]